# Proyecto 2 Steam

**Objetivo del proyecto:** El propósito de este proyecto es poder elaborar un sistema de recomendación basado en datos reales provistos por la plataforma Steam.

El sistema de recomendación sugiere títulos a un usuario en base a lo que ha comprado, jugado y lo que otros usuarios similares a él han comprado y jugado.

**Resultados del proyecto**

1. Un analisis de la información que se tiene y como podrían sacarle provecho para elaborar
un sistema de recomendación.
2. Definir una métrica o proceso que les permita evaluar su sistema de recomendación
3. Generar un sistema de recomendación, que dado la información de un usuario (ID +
Historial de compra y juegos) le recomiende otros juegos.
4. Elaborar hipótesis de que información que no se tiene podría mejorar su sistema de recomendación.

Autores:
- Omar David Hernández Aguirre  | A01383543
- Emiliano Daniel Flores Garza  | A00825175

Tec de Monterrey  
11 de mayo de 2023

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
from scipy.stats import percentileofscore
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import re
import tkinter as tk
from tkinter import messagebox


## 1. Análisis Exploratorio

Análisis exploratorio de los datasets:

- [steam_description_data.csv](#steam_description_datacsv)
- [steam_media_data.csv](#steam_media_datacsv)
- [steam_requirements_data.csv](#steam_requirements_datacsv)
- [steam_support_info.csv](#steam_support_infocsv)
- [steam_user_behavior.csv](#steam_user_behaviorcsv)
- [steam.csv](#steamcsv)
- [steamspy_tag_data.csv](#steamspy_tag_datacsv)

### Explorando Datasets

In [3]:
steam_description_df = pd.read_csv("steam_description_data.csv")
steam_media_df = pd.read_csv("steam_media_data.csv")
steam_requirements_df = pd.read_csv("steam_requirements_data.csv")
steam_support_df = pd.read_csv("steam_support_info.csv")
steam_user = pd.read_csv("steam_user_behavior.csv")
steam_df = pd.read_csv("steam.csv")
steamspy_tag_df = pd.read_csv("steamspy_tag_data.csv")

##### `steam_description_data.csv`

In [14]:
steam_description_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27334 entries, 0 to 27333
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   steam_appid           27334 non-null  int64 
 1   detailed_description  27334 non-null  object
 2   about_the_game        27334 non-null  object
 3   short_description     27334 non-null  object
dtypes: int64(1), object(3)
memory usage: 854.3+ KB


In [15]:
steam_description_df.head()

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


In [16]:
steam_description_df.describe()

,steam_appid
count,2.733400e+04
mean,5.982886e+05
std,2.512113e+05
min,1.000000e+01
25%,4.030925e+05
50%,6.019650e+05
75%,8.011750e+05
max,1.069460e+06


In [17]:
steam_description_df.describe(include='all')

,steam_appid,detailed_description,about_the_game,short_description
count,2.733400e+04,27334,27334,27334
unique,NaN,27315,27315,27204
top,NaN,"The 58th year of Shouwa, early summer.<br>It’s...","The 58th year of Shouwa, early summer.<br>It’s...",Minimal physical puzzle with explosions
freq,NaN,3,3,12
mean,5.982886e+05,NaN,NaN,NaN
std,2.512113e+05,NaN,NaN,NaN
min,1.000000e+01,NaN,NaN,NaN
25%,4.030925e+05,NaN,NaN,NaN
50%,6.019650e+05,NaN,NaN,NaN
75%,8.011750e+05,NaN,NaN,NaN


In [18]:
print(f"Duplicados: {steam_description_df.duplicated().sum()}")

Duplicados: 0


##### `steam_media_data.csv`

In [19]:
steam_media_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27332 entries, 0 to 27331
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   steam_appid   27332 non-null  int64 
 1   header_image  27332 non-null  object
 2   screenshots   27332 non-null  object
 3   background    27332 non-null  object
 4   movies        25641 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


In [20]:
steam_media_df.head()

,steam_appid,header_image,screenshots,background,movies
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN
1,20,https://steamcdn-a.akamaihd.net/steam/apps/20/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN
2,30,https://steamcdn-a.akamaihd.net/steam/apps/30/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/30/...,NaN
3,40,https://steamcdn-a.akamaihd.net/steam/apps/40/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN
4,50,https://steamcdn-a.akamaihd.net/steam/apps/50/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN


In [21]:
steam_media_df.describe()

,steam_appid
count,2.733200e+04
mean,5.982850e+05
std,2.512123e+05
min,1.000000e+01
25%,4.030850e+05
50%,6.019650e+05
75%,8.011650e+05
max,1.069460e+06


In [22]:
steam_media_df.describe(include='all')

,steam_appid,header_image,screenshots,background,movies
count,2.733200e+04,27332,27332,27332,25641
unique,NaN,27332,27332,27332,25639
top,NaN,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 5968, 'name': 'X Superbox Trailer', 't..."
freq,NaN,1,1,1,3
mean,5.982850e+05,NaN,NaN,NaN,NaN
std,2.512123e+05,NaN,NaN,NaN,NaN
min,1.000000e+01,NaN,NaN,NaN,NaN
25%,4.030850e+05,NaN,NaN,NaN,NaN
50%,6.019650e+05,NaN,NaN,NaN,NaN
75%,8.011650e+05,NaN,NaN,NaN,NaN


In [23]:
print(f"Duplicados: {steam_media_df.duplicated().sum()}")

Duplicados: 0


##### `steam_requirements_data.csv`

In [24]:
steam_requirements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27319 entries, 0 to 27318
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   steam_appid         27319 non-null  int64 
 1   pc_requirements     27319 non-null  object
 2   mac_requirements    27319 non-null  object
 3   linux_requirements  27319 non-null  object
 4   minimum             27314 non-null  object
 5   recommended         14134 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.3+ MB


In [25]:
steam_requirements_df.head()

,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
3,40,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
4,50,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN


In [26]:
steam_requirements_df.describe()

,steam_appid
count,2.731900e+04
mean,5.984052e+05
std,2.511286e+05
min,1.000000e+01
25%,4.031950e+05
50%,6.020400e+05
75%,8.012050e+05
max,1.069460e+06


In [27]:
steam_requirements_df.describe(include="all")

,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
count,2.731900e+04,27319,27319,27319,27314,14134
unique,NaN,25411,8086,5300,25132,12318
top,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],OS: Windows 7,Requires a 64-bit processor and operating system
freq,NaN,134,16101,18972,138,817
mean,5.984052e+05,NaN,NaN,NaN,NaN,NaN
std,2.511286e+05,NaN,NaN,NaN,NaN,NaN
min,1.000000e+01,NaN,NaN,NaN,NaN,NaN
25%,4.031950e+05,NaN,NaN,NaN,NaN,NaN
50%,6.020400e+05,NaN,NaN,NaN,NaN,NaN
75%,8.012050e+05,NaN,NaN,NaN,NaN,NaN


##### `steam_support_info.csv`

In [28]:
steam_support_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27136 entries, 0 to 27135
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   steam_appid    27136 non-null  int64 
 1   website        18015 non-null  object
 2   support_url    16479 non-null  object
 3   support_email  23500 non-null  object
dtypes: int64(1), object(3)
memory usage: 848.1+ KB


In [29]:
steam_support_df.head()

,steam_appid,website,support_url,support_email
0,10,NaN,http://steamcommunity.com/app/10,NaN
1,30,http://www.dayofdefeat.com/,NaN,NaN
2,50,NaN,https://help.steampowered.com,NaN
3,70,http://www.half-life.com/,http://steamcommunity.com/app/70,NaN
4,80,NaN,http://steamcommunity.com/app/80,NaN


In [30]:
steam_support_df.describe()

,steam_appid
count,2.713600e+04
mean,6.014279e+05
std,2.483102e+05
min,1.000000e+01
25%,4.062875e+05
50%,6.038550e+05
75%,8.021000e+05
max,1.069460e+06


In [31]:
steam_support_df.describe(include='all')

,steam_appid,website,support_url,support_email
count,2.713600e+04,18015,16479,23500
unique,NaN,15320,11100,14174
top,NaN,https://www.choiceofgames.com/,https://bigfishgames.custhelp.com/app/home,info@bigfishgames.com
freq,NaN,130,172,202
mean,6.014279e+05,NaN,NaN,NaN
std,2.483102e+05,NaN,NaN,NaN
min,1.000000e+01,NaN,NaN,NaN
25%,4.062875e+05,NaN,NaN,NaN
50%,6.038550e+05,NaN,NaN,NaN
75%,8.021000e+05,NaN,NaN,NaN


##### `steam_user_behavior.csv`

In [33]:
steam_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   UserID  200000 non-null  int64  
 1   Name    200000 non-null  object 
 2   Action  200000 non-null  object 
 3   Value   200000 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


In [34]:
steam_user.head()

,UserID,Name,Action,Value
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [35]:
steam_user.describe()

,UserID,Value
count,2.000000e+05,200000.000000
mean,1.036559e+08,17.874384
std,7.208074e+07,138.056952
min,5.250000e+03,0.100000
25%,4.738420e+07,1.000000
50%,8.691201e+07,1.000000
75%,1.542309e+08,1.300000
max,3.099031e+08,11754.000000


In [36]:
steam_user.describe(include='all')

,UserID,Name,Action,Value
count,2.000000e+05,200000,200000,200000.000000
unique,NaN,5155,2,NaN
top,NaN,Dota 2,purchase,NaN
freq,NaN,9682,129511,NaN
mean,1.036559e+08,NaN,NaN,17.874384
std,7.208074e+07,NaN,NaN,138.056952
min,5.250000e+03,NaN,NaN,0.100000
25%,4.738420e+07,NaN,NaN,1.000000
50%,8.691201e+07,NaN,NaN,1.000000
75%,1.542309e+08,NaN,NaN,1.300000


##### `steam.csv`

In [37]:
steam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   steam_appid       27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27075 non-null  object 
 5   publisher         27075 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

In [38]:
steam_df.head()

,steam_appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [39]:
steam_df.describe()

,steam_appid,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price
count,2.707500e+04,27075.000000,27075.000000,27075.000000,2.707500e+04,27075.000000,27075.000000,27075.00000,27075.000000
mean,5.962035e+05,0.981127,0.354903,45.248864,1.000559e+03,211.027147,149.804949,146.05603,6.078193
std,2.508942e+05,0.136081,2.406044,352.670281,1.898872e+04,4284.938531,1827.038141,2353.88008,7.874922
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.00000,0.000000
25%,4.012300e+05,1.000000,0.000000,0.000000,6.000000e+00,2.000000,0.000000,0.00000,1.690000
50%,5.990700e+05,1.000000,0.000000,7.000000,2.400000e+01,9.000000,0.000000,0.00000,3.990000
75%,7.987600e+05,1.000000,0.000000,23.000000,1.260000e+02,42.000000,0.000000,0.00000,7.190000
max,1.069460e+06,1.000000,18.000000,9821.000000,2.644404e+06,487076.000000,190625.000000,190625.00000,421.990000


In [40]:
steam_df.describe(include='all')

,steam_appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
count,2.707500e+04,27075,27075,27075.000000,27075,27075,27075,27075.000000,27075,27075,27075,27075.000000,2.707500e+04,27075.000000,27075.000000,27075.00000,27075,27075.000000
unique,NaN,27033,2619,NaN,17113,14354,7,NaN,3333,1552,6423,NaN,NaN,NaN,NaN,NaN,13,NaN
top,NaN,Dark Matter,2018-07-13,NaN,Choice of Games,Big Fish Games,windows,NaN,Single-player,Action;Indie,Action;Indie;Casual,NaN,NaN,NaN,NaN,NaN,0-20000,NaN
freq,NaN,3,64,NaN,94,212,18398,NaN,6110,1852,845,NaN,NaN,NaN,NaN,NaN,18596,NaN
mean,5.962035e+05,NaN,NaN,0.981127,NaN,NaN,NaN,0.354903,NaN,NaN,NaN,45.248864,1.000559e+03,211.027147,149.804949,146.05603,NaN,6.078193
std,2.508942e+05,NaN,NaN,0.136081,NaN,NaN,NaN,2.406044,NaN,NaN,NaN,352.670281,1.898872e+04,4284.938531,1827.038141,2353.88008,NaN,7.874922
min,1.000000e+01,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000e+00,0.000000,0.000000,0.00000,NaN,0.000000
25%,4.012300e+05,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,6.000000e+00,2.000000,0.000000,0.00000,NaN,1.690000
50%,5.990700e+05,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,7.000000,2.400000e+01,9.000000,0.000000,0.00000,NaN,3.990000
75%,7.987600e+05,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,23.000000,1.260000e+02,42.000000,0.000000,0.00000,NaN,7.190000


##### `steamspy_tag_data.csv`

In [41]:
steamspy_tag_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29022 entries, 0 to 29021
Columns: 372 entries, steam_appid to e_sports
dtypes: int64(372)
memory usage: 82.4 MB


In [42]:
steamspy_tag_df.head()

,steam_appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550
1,20,0,71,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,122,0,0,0
3,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,77,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
steamspy_tag_df.describe()

,steam_appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
count,2.902200e+04,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,...,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000
mean,5.952577e+05,0.183826,0.167011,0.137689,3.646992,0.248053,0.005789,0.060092,0.233547,0.093791,...,0.134140,0.049962,0.017573,0.158742,0.023637,0.092034,0.761698,0.016470,3.314382,0.574426
std,2.521478e+05,7.916178,5.114638,3.228531,47.377053,7.160597,0.455944,1.139116,4.750858,3.142058,...,5.722873,1.335872,0.934081,7.371732,1.002216,5.736370,24.977839,0.892563,104.515689,56.920088
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.997825e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.994700e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.987275e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.069460e+06,954.000000,564.000000,253.000000,5626.000000,641.000000,68.000000,70.000000,365.000000,221.000000,...,552.000000,109.000000,106.000000,806.000000,117.000000,746.000000,2697.000000,78.000000,12338.000000,8406.000000


In [44]:
steamspy_tag_df.describe(include="all")

,steam_appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
count,2.902200e+04,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,...,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000,29022.000000
mean,5.952577e+05,0.183826,0.167011,0.137689,3.646992,0.248053,0.005789,0.060092,0.233547,0.093791,...,0.134140,0.049962,0.017573,0.158742,0.023637,0.092034,0.761698,0.016470,3.314382,0.574426
std,2.521478e+05,7.916178,5.114638,3.228531,47.377053,7.160597,0.455944,1.139116,4.750858,3.142058,...,5.722873,1.335872,0.934081,7.371732,1.002216,5.736370,24.977839,0.892563,104.515689,56.920088
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.997825e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.994700e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.987275e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.069460e+06,954.000000,564.000000,253.000000,5626.000000,641.000000,68.000000,70.000000,365.000000,221.000000,...,552.000000,109.000000,106.000000,806.000000,117.000000,746.000000,2697.000000,78.000000,12338.000000,8406.000000


### Análisis

Una vez vistas las características de cada uno de los datasets, podemos elegir aquellas que serán de más utilidad para el sistema de recomendación, o crear nuevas características a partir de información de los diferentes datasets.

Podemos ver que el dataset con más relevancia es el `steam_user_behavior.csv` ya que podemos utilizar las horas de juego del usuario para recomendarle juegos similares en un futuro. Pero, vamos a reestructurar el dataset en un dataframe que contenga sólo la información relevante.

In [80]:
#Creamos las columnas "purchase" y "play"
purchase_data = steam_user[steam_user['Action'] == 'purchase'].copy()
play_data = steam_user[steam_user['Action'] == 'play'].copy()

#Agregamos la columna "_purchase" y "_play"
purchase_data['Value_purchase'] = 1
play_data['Value_play'] = play_data['Value']

#Combinamos los datos
merged_data = pd.merge(purchase_data, play_data, on=['UserID', 'Name'], how='left')
merged_data['Value_play'].fillna(0, inplace=True)

#Eliminamos columnas innecesarias
merged_data.drop(['Action_x', 'Action_y', 'Value_x', 'Value_y'], axis=1, inplace=True)
steam_user_clean = merged_data.drop('Value_purchase', axis=1)

play_data = steam_user[steam_user['Action'] == 'play'].copy()

print(steam_user_clean.head())

      UserID                        Name  Value_play
0  151603712  The Elder Scrolls V Skyrim       273.0
1  151603712                   Fallout 4        87.0
2  151603712                       Spore        14.9
3  151603712           Fallout New Vegas        12.1
4  151603712               Left 4 Dead 2         8.9


Una vez teniendo únicamente los datos relevantes, podemos calcular una nueva característica `Rating` que indicará el rating del videojuego por cada usuario dependiendo de las horas jugadas. Es decir, cada usuario tendrá un `Rating` de todos los juegos que ha jugado; y el video con más horas de juego tendrá más rating, mientras que el videojuego con menos horas de juego tendrá menor rating.

In [89]:
#Calculamos el percentil de las horas jugadas
percentiles = steam_user_clean['Value_play'].apply(lambda x: percentileofscore(steam_user_clean['Value_play'], x))

#Mapeamos los percentiles a un rango de 0 a 100
rating = percentiles.apply(lambda x: int(x * 100 / 100))

#Agregamos la columna "Rating" al DataFrame
steam_user_clean['Rating'] = rating
steam_user_clean.head()

,UserID,Name,Value_play,Rating
0,151603712,The Elder Scrolls V Skyrim,273.0,98
1,151603712,Fallout 4,87.0,95
2,151603712,Spore,14.9,84
3,151603712,Fallout New Vegas,12.1,82
4,151603712,Left 4 Dead 2,8.9,79


También podemos ver que algunos datasets comparten la columna `steam_appid`. Esto da a entender que hay una asociación entre los diferentes datasets mediante esta columna (e.g. una asociación en una base de datos relacional). Por lo que, podríamos juntar los datasets en un sólo dataframe.

In [6]:
#Cambiamos el nombre de la columna "appid" a "steam_appid"
steam_df.rename(columns={"appid": "steam_appid"}, inplace=True)
steamspy_tag_df.rename(columns={"appid": "steam_appid"}, inplace=True)

In [7]:
#Combinamos los datasets de la info de los videojuegos
games_df = steam_description_df.merge(steam_media_df, on="steam_appid").merge(steam_requirements_df, on="steam_appid").merge(steam_support_df, on="steam_appid").merge(steam_df, on="steam_appid").merge(steamspy_tag_df, on="steam_appid")

In [94]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26868 entries, 0 to 26867
Columns: 404 entries, steam_appid to e_sports
dtypes: float64(1), int64(379), object(24)
memory usage: 83.0+ MB


No todas las características nos son útiles para el sistema. Las características que creemos son pertinentes para hacer las recomendaciones son:
- `steam_appid`
- `short_description`
- `name`
- `release_date`
- `developer`
- `platforms`
- `categories`
- `genres`
- `steamspy_tags`
- `price`

In [96]:
#Creamos un nuevo DataFrame con columnas seleccionadas
games_df_clean = games_df[['steam_appid', 'short_description', 'name','release_date', 'developer', 'platforms','categories', 'genres', 'steamspy_tags','price']].copy()

In [97]:
games_df_clean.head()

,steam_appid,short_description,name,release_date,developer,platforms,categories,genres,steamspy_tags,price
0,10,Play the world's number 1 online action game. ...,Counter-Strike,2000-11-01,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,7.19
1,30,Enlist in an intense brand of Axis vs. Allied ...,Day of Defeat,2003-05-01,Valve,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,3.99
2,50,Return to the Black Mesa Research Facility as ...,Half-Life: Opposing Force,1999-11-01,Gearbox Software,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,3.99
3,70,Named Game of the Year by over 50 publications...,Half-Life,1998-11-08,Valve,windows;mac;linux,Single-player;Multi-player;Online Multi-Player...,Action,FPS;Classic;Action,7.19
4,80,"With its extensive Tour of Duty campaign, a ne...",Counter-Strike: Condition Zero,2004-03-01,Valve,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,Action;FPS;Multiplayer,7.19


In [98]:
games_df_clean.rename(columns={"name": "Name"}, inplace=True)

Por último, vamos limpiar la columna `Name` quitando dobles espacios, caracteres no alfabéticos y capitalizando la primera letra de cada palabra. Guardaremos los dataframes resultantes en archivos CSV (`games_df_clean.csv` y `steam_user_clean`)

In [99]:
#Eliminamos caracteres no alfabéticos de los nombres de los videojuegos
steam_user_clean['Name'] = steam_user_clean['Name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
games_df_clean['Name'] = games_df_clean['Name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

#Eliminamos dobles espacios
steam_user_clean['Name'] = steam_user_clean['Name'].apply(lambda x: ' '.join(x.split()))
games_df_clean['Name'] = games_df_clean['Name'].apply(lambda x: ' '.join(x.split()))

#Captializamos la primera letra de cada palabra
steam_user_clean['Name'] = steam_user_clean['Name'].apply(lambda x: x.title())
games_df_clean['Name'] = games_df_clean['Name'].apply(lambda x: x.title())


steam_user_clean.to_csv('steam_user_clean.csv', index=False)
games_df_clean.to_csv('games_df_clean.csv', index=False)

In [100]:
#Combinamos los DataFrames basándose en la columna "Name"
df = steam_user_clean.merge(games_df_clean, on='Name', how='left')
df.dropna(inplace=True)
#Reorganizamos columnas
df = df[[col for col in df.columns if col != 'Rating'] + ['Rating']]
df.head()

,UserID,Name,Value_play,steam_appid,short_description,release_date,developer,platforms,categories,genres,steamspy_tags,price,Rating
0,151603712,The Elder Scrolls V Skyrim,273.0,72850.0,EPIC FANTASY REBORN The next chapter in the hi...,2011-11-10,Bethesda Game Studios,windows,Single-player;Steam Achievements;Steam Trading...,RPG,Open World;RPG;Fantasy,9.99,98
1,151603712,Fallout 4,87.0,377160.0,"Bethesda Game Studios, the award-winning creat...",2015-11-09,Bethesda Game Studios,windows,Single-player;Steam Achievements;Full controll...,RPG,Open World;Post-apocalyptic;Exploration,19.99,95
2,151603712,Spore,14.9,17390.0,"From Single Cell to Galactic God, evolve your ...",2008-12-19,Maxis™,windows,Single-player,Simulation,Simulation;Sandbox;Open World,14.99,84
3,151603712,Fallout New Vegas,12.1,22380.0,Welcome to Vegas. New Vegas. Enjoy your stay!,2010-10-21,Obsidian Entertainment,windows,Single-player;Steam Achievements;Partial Contr...,Action;RPG,Open World;RPG;Post-apocalyptic,7.99,82
4,151603712,Left 4 Dead 2,8.9,550.0,"Set in the zombie apocalypse, Left 4 Dead 2 (L...",2009-11-19,Valve,windows;mac;linux,Single-player;Multi-player;Co-op;Steam Achieve...,Action,Zombies;Co-op;FPS,7.19,79


## 2. Métrica para evaluación del sistema

## 3. Sistema de recomendación

In [13]:
#Seleccionamos las columnas relevantes para el análisis
columns = ['UserID', 'Name', 'genres', 'categories', 'platforms', 'price', 'Rating']

#Filtramos el dataset con las columnas seleccionadas
df_filtered = df[columns]

#Limpiamos y preprocesamos los datos de las columnas###############################################################################################

#Combinamos las columnas de texto en una sola columna para representar los datos de los usuarios
df_filtered['combined_features'] = df_filtered['genres'] + ' ' + df_filtered['categories'] + ' ' + df_filtered[
    'platforms'] + ' ' + df_filtered['price'].astype(str)

#Creamos una matriz TF-IDF para representar los datos de los usuarios
tfidf = TfidfVectorizer()
user_matrix = tfidf.fit_transform(df_filtered['combined_features'])

#Aplicamos el algoritmo de clustering (K-means)
k = 20  # Número de clusters deseados
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(user_matrix)

#Agregamos las etiquetas de clusters al dataframe original
df_filtered['Cluster'] = kmeans.labels_

#Función para obtener las recomendaciones para un usuario específico
def obtener_recomendaciones(usuario):
    user_cluster = df_filtered.loc[df_filtered['UserID'] == usuario, 'Cluster'].values[0]

    #Filtramos los videojuegos pertenecientes al mismo cluster del usuario
    cluster_games = df_filtered.loc[df_filtered['Cluster'] == user_cluster]

    #Excluimos los juegos que el usuario ya ha jugado
    played_games = df_filtered.loc[df_filtered['UserID'] == usuario, 'Name'].unique()
    recommended_games = cluster_games.loc[~cluster_games['Name'].isin(played_games), 'Name'].unique()

    return recommended_games[:5]


#Función para manejar el evento del botón
def obtener_recomendaciones_usuario():
    nombre_usuario = entry_usuario.get()

    #Obtenemos el ID del usuario
    usuario = df_filtered.loc[df_filtered['UserID'] == int(nombre_usuario), 'UserID'].unique()
    if len(usuario) == 0:
        messagebox.showerror("Error", "Usuario no encontrado.")
        return

    usuario_id = usuario[0]

    #Obtenemos las recomendaciones
    recomendaciones = obtener_recomendaciones(usuario_id)

    #Mostramos las recomendaciones de videojuegos en una ventana emergente
    messagebox.showinfo("Recomendaciones", "\n".join(recomendaciones))


#Creamos la ventana
ventana = tk.Tk()
ventana.title("Recomendaciones de videojuegos")

#Creamos un campo de entrada de texto
entry_usuario = tk.Entry(ventana)
entry_usuario.pack()

#Creamos un botón para obtener las recomendaciones
boton_obtener_recomendaciones = tk.Button(ventana, text="Obtener Recomendaciones", command=obtener_recomendaciones_usuario)
boton_obtener_recomendaciones.pack()

#Ejecutamos el bucle principal de la ventana
ventana.mainloop()


/var/folders/m9/m_cd57bd39j6t0623_qd7s2r0000gn/T/ipykernel_79015/4154567009.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['combined_features'] = df_filtered['genres'] + ' ' + df_filtered['categories'] + ' ' + df_filtered[
/Users/davidhdz/anaconda3/envs/machine_learning/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/var/folders/m9/m_cd57bd39j6t0623_qd7s2r0000gn/T/ipykernel_79015/4154567009.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

## 4. Hipótesis de mejora
Elaborar una hipótesis de qué información no se tiene y podría mejorar el sistema de recomendación